#### Objective:
Create a directional, weighted graph from the provided edge list and compute the total upstream throughput at each node. 

In [ ]:
#Import packages
import pandas as pd
import networkx as nx
import arcpy

In [ ]:
#Read in the edge list and add edges
df_edgeList = pd.read_csv('../scratch/SingleConnect_EdgeList.csv',
                          dtype={'From_node':'str','To_node':'str'})
df_edgeList.head()

In [ ]:
#Create the empty directional graph
G = nx.DiGraph()

In [ ]:
#Add the edges
for i,r in df_edgeList.iterrows():
    G.add_edge(r.From_node,r.To_node,cost=r.Site_yield_scf_h)

In [ ]:
nodeCosts = []
#Iterate through each to_node
for from_node in df_edgeList.From_node.unique():
    #Reset the cost variable
    total_yield = 0
    #Get the upstream nodes for the current node
    up_nodes = nx.ancestors(G,from_node)
    #Append the node itself
    up_nodes.add(from_node)
    #Iterate through the to_node's ancestors
    for up_node in up_nodes:
        #Accumulate the total yield
        total_yield += df_edgeList.query("From_node == '{}'".format(up_node))['Site_yield_scf_h'].sum()
    df_edgeList.loc[df_edgeList.From_node == from_node,'SumYield'] = total_yield

In [ ]:
#Compute pipe diameter from throughput
df_edgeList['PipeSize_in'] = df_edgeList['SumYield']**0.394*0.051
#Compute pipe cost from throughput 
df_edgeList['PipeCost'] = df_edgeList['PipeSize_in']**0.485*10829 
#Compute return
df_edgeList['Return_15yr'] = df_edgeList['PipeCost'] / (df_edgeList['SumYield'] * 15)

In [ ]:
#Save edge list to a filename
edgeCSV_filename =  '../scratch/SingleConnect_EdgeList2.csv'
df_edgeList[['From_node','Site_yield_scf_h', 'SumYield', 'PipeSize_in',
            'PipeCost', 'Return_15yr']].to_csv(edgeCSV_filename,index=False)

In [ ]:
#Join edge list to edge features
#Create a table from the CSV
edge_table = arcpy.management.MakeTableView(edgeCSV_filename,'edge_table')
edge_features = '..\\OptimizeViewer\\SingleConnect.gdb\LCP_edges_SingleConnect'
#Join to the edgeFC
arcpy.management.JoinField(edge_features,'Edge_ID',
                           edge_table,
                           'From_node',
                           ['Site_yield_scf_h', 'SumYield', 'PipeSize_in','PipeCost', 'Return_15yr'])